In [113]:
import glob
import re
import regex
import pandas as pd
import string


<h1>  Dataframe Columns </h1>

MMSID<br>
editionTitle<br>
editor <br>
editor_date <br>
genre<br>
language<br>
termsOfAddress<br>
numberOfPages <br>
physicalDescription<br>
place<br>
publisher<br>
referencedBy<br>
shelfLocator<br>
editionSubTitle<br>
volumeTitle<br>
year<br>
volumeId<br>
metsXML<br>
permanentURL<br>
publisherPersons<br>
volumeNum<br>
letters<br>
part0<br>
editionNum<br>
supplementTitle<br>
supplementSubTitle<br>
supplementsTo<br>
numberOfVolumes<br>
term<br>
definition<br>
relatedTerms<br>
header<br>
startsAt<br>
endsAt<br>
numberOfTerms<br>
numberOfWords<br>
positionPage<br>
typeTerm<br>
altoXML<br>

In [77]:
data_path = "../data/eb07-v1.2-TXT/"

#---------------------hardcoded columns to match dataframe structure---------------#
parsed_data = []
MMSID = "9910796273804340"
editor = "Stewart, Dugald"
editor_date = "1753-1828"
genre = "encyclopedia"
language = "eng"
termsOfAddress = "Sir"
numberOfPages = "184"
physicalDescription = "21 v. in 22 ; 4to."
editionSubTitle = "0.0"
place = "Edinburgh"
publisher = "A. & C. Black"
referencedBy = "0.0"
shelfLocator = "EB.15"
volumeId = "192547789"
metsXML = "192547789-mets.xml"
permanentURL = "https://digital.nls.uk/192547789"
publisherPersons = []
part = "0"
edTitle = "Seventh edition, General index"
header = "Not specified"
place = "Edinburgh"
positionPage = 0 #add 1 for each term in a file
numberOfPages = "1" #find similar term and mash into df
volumeTitle = "Encyclopaedia Britannica"
year = "1842" 
part = "Not specified"
supplementTitle = "Not specified"
supplementsTo = "Not specified"
relatedTerms = "Not specified"
numberOfVolumes = "20" 
supplementSubTitle = "Not specified"
numberOfTerms = "Not specified"
#----------------------------------------------------------------------------#
letters = "" 
volumeNum = "" #parse out from file name
typeTerm = "Topic" #0 is a term, 1 is an article
startsAt = "Not specified" #parse from file
endsAt = "Not specified" #same as starts at

#for finding volume and letter
volumeRegex = '/[a-z][0-9]+/'
#for removing punctuation but leaving hyphens
punctuationHyphen = string.punctuation.replace('-', "")

for file in sorted(glob.glob(data_path + "*/*.txt")):
    #get file namefrom file path
    altoXML = file.removeprefix("../data/")
    
    #usesful for seeing how far along the data processing is going
    #print(altoXML)
    
    #get volume and letter from file path
    volume_letter = file.removeprefix(data_path).split("/")[0]
    volumeNum = volume_letter[1:]
    letters = volume_letter[0]
    

    with open (file, 'r') as currFile:
        text = currFile.read()
        text = text.split("=+")
        
        #get data of location in EB
        ed_info = re.findall("\[\d+\:\d+\:\d+\]", text[1])[0].replace("[", "").replace("]", "")
        ed_info = ed_info.split(":")
        editionNum = ed_info[0]
        volumeNum = ed_info[1]
        startsAt = ed_info[2]
        endsAt = ed_info[2]
        
        #get each line from the text file
        lines = text[2].split('\n')
        lines = [x for x in lines if x]
        
        #for finding new terms
        capital_pattern = '^[A-Z\s\']+(?![a-z])'
        
        #BAAL 
        #Baal-perith
        #
        
        term = ""
        definition = ""
        positionPage = 0
        #for each line in the file decide if it is a new term, or a new definition, or continuing definition
        for line in lines:
            print(line)
            new_term = re.match(capital_pattern, line)
            #if there is a new term and the new term is not a single letter
            if(new_term and new_term.group().strip() != term):
                #if a term already exists in the file then save the old term and start a new term
                if(term != ""):
                    #calculate the remaining columns
                    numberOfWords = len(definition.split())
                    if(numberOfWords > 1000):
                        typeTerm = "Article"
                    else:
                        typeTerm = "Topic"
                    positionPage += 1
                    #clean defintion
                    if definition[:2] == ", " or definition[:2] == ". ":
                        definition = definition[2:]
                    parsed_data.append([term, definition, MMSID, edTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages,
                            physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle,
                            volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum,
                            letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes,
                            relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML])
                #start new term    
                term = new_term.group().strip()
                definition += line.replace(term, "")
                
            elif(line.split(',')[0].split("-")[0].upper() == term.split("-")[0]):
                
                #calculate the remaining columns
                positionPage += 1
                numberOfWords = len(definition.split())
                if(numberOfWords > 1000):
                    typeTerm = "Article"
                else:
                    typeTerm = "Topic"
                #clean defintion
                if definition[:2] == ", " or definition[:2] == ". ":
                    definition = definition[2:]
                parsed_data.append([term, definition, MMSID, edTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages,
                            physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle,
                            volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum,
                            letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes,
                            relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML])
                #same term but different definition 
                #replace is for a specific edge case so that serialization of the KG does not break
                term = line.split(',')[0].upper()
                #set(string.punctuation) - set('-')
                term = term.translate(str.maketrans('', '', punctuationHyphen))
                definition = "".join(line.split(",")[1:]).strip()
            else:
                #continue definition of current term bc it is a multiline definition
                definition += line
            
        #add last term in file
        #calculate the remaining columns
        numberOfWords = len(definition.split())
        if(numberOfWords > 1000):
            typeTerm = "Article"
        else:
            typeTerm = "Topic"
        positionPage += 1
        #clean defintion
        if definition[:2] == ", " or definition[:2] == ". ":
            definition = definition[2:]
        parsed_data.append([term, definition, MMSID, edTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages,
                            physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle,
                            volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum,
                            letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes,
                            relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML])
        
    #create dataframe with data and columns        
    df = pd.DataFrame(parsed_data, columns=["term", "definition", "MMSID", "edTitle", "editor", "editor_date", "genre", "language", "termsOfAddress", "numberOfPages",
                            "physicalDescription", "place", "publisher", "referencedBy", "shelfLocator", "editionSubTitle",
                            "volumeTitle", "year", "volumeId", "metsXML", "permanentURL", "publisherPersons", "volumeNum",
                            "letters", "part", "editionNum", "supplementTitle", "supplementSubTitle", "supplementsTo", "numberOfVolumes",
                            "relatedTerms", "header", "startsAt", "endsAt", "numberOfTerms", "numberOfWords", "positionPage", "typeTerm", "altoXML"])



A. The first letter of the alphabet in every known language, the Ethiopie or Abyssinian alone excepted, in which it is the thirteenth. Of the sixteen elementary sounds of the human voice, that which is represented by this initial letter is the simplest, and requires the least exertion of the organs to produce it; for its enunciation is effected by merely opening the mouth, and breathing, so that the air propelled through the glottis may resound audibly in the cavity of the mouth and nostrils. Hence this sound is remarkable for its universality as well as simplicity. Many of the lower animals possess the capacity of uttering it, as every one must be sensible who has attended to their distinguishing cries, in all, or at least in many of which, it may be easily recognised. It is also the basis, so to speak, of vocality; for, on attentive examination, it will be found that the other vowels are little more than labial, lingual, dental, or palatal modifications of this primary, universal, an

KeyboardInterrupt: 

In [123]:
data_path = "../data/eb07-v1.2-TXT/"

#---------------------hardcoded columns to match dataframe structure---------------#
parsed_data = []
MMSID = "9910796273804340"
editor = "Stewart, Dugald"
editor_date = "1753-1828"
genre = "encyclopedia"
language = "eng"
termsOfAddress = "Sir"
numberOfPages = "184"
physicalDescription = "21 v. in 22 ; 4to."
editionSubTitle = "0.0"
place = "Edinburgh"
publisher = "A. & C. Black"
referencedBy = "0.0"
shelfLocator = "EB.15"
volumeId = "192547789"
metsXML = "192547789-mets.xml"
permanentURL = "https://digital.nls.uk/192547789"
publisherPersons = []
part = "0"
edTitle = "Seventh edition, General index"
header = "Not specified"
place = "Edinburgh"
positionPage = 0 #add 1 for each term in a file
numberOfPages = "1" #find similar term and mash into df
volumeTitle = "Encyclopaedia Britannica"
year = "1842" 
part = "Not specified"
supplementTitle = "Not specified"
supplementsTo = "Not specified"
relatedTerms = "Not specified"
numberOfVolumes = "20" 
supplementSubTitle = "Not specified"
numberOfTerms = "Not specified"
#----------------------------------------------------------------------------#
letters = "" 
volumeNum = "" #parse out from file name
typeTerm = "Topic" #0 is a term, 1 is an article
startsAt = "Not specified" #parse from file
endsAt = "Not specified" #same as starts at

#for finding volume and letter
volumeRegex = '/[a-z][0-9]+/'
#for removing punctuation but leaving hyphens
punctuationHyphen = string.punctuation.replace('-', "")

for file in sorted(glob.glob(data_path + "*/*.txt")):
    #get file namefrom file path
    altoXML = file.removeprefix("../data/")
    
    #usesful for seeing how far along the data processing is going
    #print(altoXML)
    
    #get volume and letter from file path
    volume_letter = file.removeprefix(data_path).split("/")[0]
    volumeNum = volume_letter[1:]
    letters = volume_letter[0]
    

    with open (file, 'r') as currFile:
        text = currFile.read()
        text = text.split("=+")
        
        #get data of location in EB
        ed_info = re.findall("\[\d+\:\d+\:\d+\]", text[1])[0].replace("[", "").replace("]", "")
        ed_info = ed_info.split(":")
        editionNum = ed_info[0]
        volumeNum = ed_info[1]
        startsAt = ed_info[2]
        endsAt = ed_info[2]
        
        #get each line from the text file
        lines = text[2].split('\n')
        lines = [x for x in lines if x]
        
        #for finding new terms
        capital_pattern = regex.compile(r"^[\p{Lu}\s\']+(?![a-z])")
        
        #BAAL 
        #Baal-perith
        #
        
        term = ""
        definition = ""
        positionPage = 0
        new_term = capital_pattern.match(lines[0])
        #if there is a new term and the new term is not a single letter
        if(new_term):
            #if a term already exists in the file then save the old term and start a new term
            #start new term    
            term = new_term.group().strip()
            # print(term)
            definition += lines[0][len(term):]
        #if term is not capitalized like in eb07-v1.2-TXT/s20/kp-eb0720-010203-0116-v1.txt
        if(term == ""):
            print(altoXML)
            print(lines[0])
            term = lines[0].split(",")[0]
            #if there is no comma separating term
            if(len(term.split(" ")) > 4):
                term = lines[0].split(" ")[0].replace(".", "").replace(",", "")
        #for each line in the file decide if it is a new term, or a new definition, or continuing definition
        for line in lines[1:]:
            if(term == line.split(',')[0].upper() or term == line.split(',')[0].split("-")[0].upper()):
                # print(term)          
                # print(line)
                # print(line.split(',')[0].upper())
                #calculate the remaining columns
                positionPage += 1
                numberOfWords = len(definition.split())
                if(numberOfWords > 1000):
                    typeTerm = "Article"
                else:
                    typeTerm = "Topic"
                #clean defintion
                if definition[:2] == ", " or definition[:2] == ". ":
                    definition = definition[2:]
                parsed_data.append([term, definition, MMSID, edTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages,
                            physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle,
                            volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum,
                            letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes,
                            relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML])
                #same term but different definition 
                #replace is for a specific edge case so that serialization of the KG does not break
                term = line.split(',')[0].upper()
                if(term == ""):
                    print("new")
                    print(altoXML)
                    print(line)
                # print("new:" + term)
                #set(string.punctuation) - set('-')
                term = term.translate(str.maketrans('', '', punctuationHyphen))
                definition = "".join(line.split(",")[1:]).strip()
            else:
                #continue definition of current term bc it is a multiline definition
                definition += line
            
        #add last term in file
        #calculate the remaining columns
        numberOfWords = len(definition.split())
        if(numberOfWords > 1000):
            typeTerm = "Article"
        else:
            typeTerm = "Topic"
        positionPage += 1
        #clean defintion
        if definition[:2] == ", " or definition[:2] == ". ":
            definition = definition[2:]
        parsed_data.append([term, definition, MMSID, edTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages,
                            physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle,
                            volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum,
                            letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes,
                            relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML])
        
    #create dataframe with data and columns        
    df = pd.DataFrame(parsed_data, columns=["term", "definition", "MMSID", "edTitle", "editor", "editor_date", "genre", "language", "termsOfAddress", "numberOfPages",
                            "physicalDescription", "place", "publisher", "referencedBy", "shelfLocator", "editionSubTitle",
                            "volumeTitle", "year", "volumeId", "metsXML", "permanentURL", "publisherPersons", "volumeNum",
                            "letters", "part", "editionNum", "supplementTitle", "supplementSubTitle", "supplementsTo", "numberOfVolumes",
                            "relatedTerms", "header", "startsAt", "endsAt", "numberOfTerms", "numberOfWords", "positionPage", "typeTerm", "altoXML"])



eb07-v1.2-TXT/s20/kp-eb0720-007502-0089-v1.txt
See Bangog, a small island in the Eastern Seas, near the east coast of Borneo. Long. 118. 24. E. Lat. 4. 18. N.
eb07-v1.2-TXT/s20/kp-eb0720-007503-0089-v1.txt
See Beeroo Isle, an island that lies off the west coast of Sumatra, and is situated principally between the first and second degrees of south latitude, and the ninety-eighth and ninety-ninth of east longitude. It is seventy miles in length by ten in average breadth, and contains a volcano, which renders it conspicuous at a distance.
eb07-v1.2-TXT/s20/kp-eb0720-008201-0096-v1.txt
Self-Love is that instinctive principle which impels every animal, rational and irrational, to preserve its life and promote its own happiness. It is very generally confounded with selfishness; but we think that the one propensity is distinct from the other. Every man loves himself, but every man is not selfish. The selfish man grasps at all immediate advantages, regardless of the consequences which his condu

In [124]:
df.head(10)


,term,definition,MMSID,edTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,...,numberOfVolumes,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML
0,A,The first letter of the alphabet in every know...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,1,1,Not specified,1440,1,Article,eb07-v1.2-TXT/a2/kp-eb0702-000101-9822-v1.txt
1,A,as an abbreviation is likewise of frequent occ...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,1,1,Not specified,135,2,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000101-9822-v1.txt
2,AA,"a river of the province of Groningen, in the k...",9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,28,1,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
3,AA,a river in the province of Overyssel. in the N...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,16,2,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
4,AA,a river of the province of Antwerp in the Neth...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,17,3,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
5,AA,a river of France rising in the Pas de Calais;...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,59,4,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
6,AA,a river in the Russian government of Courland ...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,37,5,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
7,AA,a river in the Russian province of Livonia whi...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,19,6,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
8,AAHUS,"a little town of Germany, in the circle of Wes...",9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,41,1,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000202-9835-v1.txt
9,AALBORG,one of the four sees (stifts) into which the D...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,223,1,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000203-9835-v1.txt


In [125]:
print(df.shape)
print(len(sorted(glob.glob(data_path + "*/*.txt"))))

#Number of pages in a book

(23122, 39)
20983


In [126]:
df = df.sort_values(by="term")
df.head(10)

,term,definition,MMSID,edTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,...,numberOfVolumes,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML
0,A,The first letter of the alphabet in every know...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,1,1,Not specified,1440,1,Article,eb07-v1.2-TXT/a2/kp-eb0702-000101-9822-v1.txt
1,A,as an abbreviation is likewise of frequent occ...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,1,1,Not specified,135,2,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000101-9822-v1.txt
2925,A',= 4 rf . Γ rn.^dχ . J J [ 1 + (m — 1) x 2] . ...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,185,185,Not specified,1800,1,Article,eb07-v1.2-TXT/a4/kp-eb0704-018501-1720-v1.txt
3,AA,a river in the province of Overyssel. in the N...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,16,2,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
4,AA,a river of the province of Antwerp in the Neth...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,17,3,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
5,AA,a river of France rising in the Pas de Calais;...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,59,4,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
6,AA,a river in the Russian government of Courland ...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,37,5,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
7,AA,a river in the Russian province of Livonia whi...,9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,19,6,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
2,AA,"a river of the province of Groningen, in the k...",9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,28,1,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000201-9835-v1.txt
8,AAHUS,"a little town of Germany, in the circle of Wes...",9910796273804340,"Seventh edition, General index","Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,1,...,20,Not specified,Not specified,2,2,Not specified,41,1,Topic,eb07-v1.2-TXT/a2/kp-eb0702-000202-9835-v1.txt


In [130]:
df.to_json(r'../data/final_eb_7_dataframe_clean', orient="index")

: 

In [128]:
df.loc[1816]

term                                                               AMYOT
definition             James, bishop of Auxerre and grand almoner of ...
MMSID                                                   9910796273804340
edTitle                                   Seventh edition, General index
editor                                                   Stewart, Dugald
editor_date                                                    1753-1828
genre                                                       encyclopedia
language                                                             eng
termsOfAddress                                                       Sir
numberOfPages                                                          1
physicalDescription                                   21 v. in 22 ; 4to.
place                                                          Edinburgh
publisher                                                  A. & C. Black
referencedBy                                       

In [129]:
df[df["term"] == "FAMILY X"]

,term,definition,MMSID,edTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,...,numberOfVolumes,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML


FAMILY X -> section should indicate SECT for TOPIC
AA -> 2 AA_0
A -> WAY TOO many As

In [69]:
old_df= pd.read_json('../data/final_eb_7_dataframe', orient="index") 

In [70]:
print(old_df.shape)

(13459, 39)
